<a href="https://colab.research.google.com/github/ndegwaanth/Crop-disease_prdiction/blob/main/New_Crop_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow
!pip install imagehash

In [5]:
from PIL import Image
import os
import cv2
import imagehash
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from tqdm import tqdm
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import hashlib
from PIL import Image, UnidentifiedImageError

In [9]:
def check_folder_and_extension(input_folder, output_folder):
  """
  check the existence of the folder being uploaded and the the extension
  of the dataset that are present
  """
  if not os.listdir(input_folder):
    os.makedir(output_folder)


  for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith(".png"):
      pass

## 1. Resizing Images
Resize all images to a consistent resolution (e.g., 224x224 or 256x256) to ensure uniformity in input size for the CNN.

In [7]:
def resize_images(input_folder, output_folder, target_size=(224, 224)):
    """
    Resizes all images in the input directory to the target size using OpenCV.

    Args:
        input_dir (str): Path to the directory containing input images.
        output_dir (str): Path to the directory to save resized images.
        target_size (tuple): Target size (width, height) for resizing. Default is (224, 224).
    """
    if not os.listdir(input_folder):
      os.makedir(output_folder)

    for filename in os.listdir(input_folder):
      if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Resize the image
        resize_image = cv2.resize(image, target_size)

        # saving the resize image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resize_image)

    print(f"Resizing complete! Resized images saved in {output_folder}")


## 2. Normalization
Normalize pixel values to a range of [0, 1] or [-1, 1] by dividing by 255 or using mean and standard deviation normalization.

In [10]:
def normalize_images_opencv(input_dir, output_dir, normalize_range="0_1"):
    """
    Normalizes all images in the input directory using OpenCV.

    Args:
        input_dir (str): Path to the directory containing resized images.
        output_dir (str): Path to the directory to save normalized images.
        normalize_range (str): Normalization range. Options: "0_1" (default) or "-1_1".
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image files
            # Read the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)

            # Normalize the image
            if normalize_range == "0_1":
                normalized_image = image / 255.0  # Normalize to [0, 1]
            elif normalize_range == "-1_1":
                normalized_image = (image / 127.5) - 1  # Normalize to [-1, 1]
            else:
                raise ValueError("Invalid normalize_range. Use '0_1' or '-1_1'.")

            # Convert back to uint8 for saving (optional, only if you want to visualize normalized images)
            normalized_image_uint8 = (normalized_image * 255).astype(np.uint8)

            # Save the normalized image
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, normalized_image_uint8)

    print(f"Normalization complete! Normalized images saved in {output_dir}")

## 3. Grayscale Conversion
Convert RGB images to grayscale if color information is not critical for disease detection

In [11]:
def convert_to_grayscale_opencv(input_dir, output_dir):
    """
    Converts all RGB images in the input directory to grayscale using OpenCV.

    Args:
        input_dir (str): Path to the directory containing input images.
        output_dir (str): Path to the directory to save grayscale images.
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image files
            # Read the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)

            # Convert the image to grayscale
            grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Save the grayscale image
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, grayscale_image)

    print(f"Grayscale conversion complete! Grayscale images saved in {output_dir}")

## 4. Color Space Transformation
Convert images to alternative color spaces like HSV, LAB, or YCrCb to enhance specific features like disease spots or discoloration.

In [12]:
def convert_color_space_opencv(input_dir, output_dir, color_space="HSV"):
    """
    Converts all images in the input directory to the specified color space using OpenCV.

    Args:
        input_dir (str): Path to the directory containing input images.
        output_dir (str): Path to the directory to save converted images.
        color_space (str): Target color space. Options: "HSV", "LAB", or "YCrCb". Default is "HSV".
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the color space conversion flag
    if color_space == "HSV":
        conversion_flag = cv2.COLOR_BGR2HSV
    elif color_space == "LAB":
        conversion_flag = cv2.COLOR_BGR2LAB
    elif color_space == "YCrCb":
        conversion_flag = cv2.COLOR_BGR2YCrCb
    else:
        raise ValueError("Invalid color space. Use 'HSV', 'LAB', or 'YCrCb'.")

    # Loop through all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image files
            # Read the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)

            # Convert the image to the specified color space
            converted_image = cv2.cvtColor(image, conversion_flag)

            # Save the converted image
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, converted_image)

    print(f"Color space conversion complete! Converted images saved in {output_dir}")